In [58]:
# Establish connections to the input db and variables spreadsheets

# Import modules
import os, re
import arcpy
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from dotenv import load_dotenv

# Lot the .env variables
load_dotenv()

# Set the data paths
inputs_path = os.getenv('J111_INPUTS')
spatial_path = os.getenv('J111_SPATIAL')
env_path = os.getenv('J111_ENV')

# Set path to the input variables excel workbook
vars_xlsx = os.path.join(inputs_path, 'Redland_GAM_Input_Vars.xlsx')

# Get full path of Inputs GDB
input_gdb_path = os.path.join(spatial_path, 'redland_gam_current_dev.gdb')

# Set arcpy workspace
arcpy.env.workspace = os.path.join(env_path, 'J111_redland_gam.gdb')

# Import the input DCDB Base features into a dataframe
parcels = pd.DataFrame.spatial.from_featureclass(os.path.join(input_gdb_path, 'INPUT_FC_DCDB_Base')).drop(['OBJECTID', 'SHAPE'], axis=1)

# Import the Council Property features into dataframe
land = pd.DataFrame.spatial.from_featureclass(os.path.join(input_gdb_path, 'INPUT_FC_CouncilProperty')).drop(['OBJECTID', 'SHAPE'], axis=1)

# Inspect dataframe
parcels.head()

,segpar,lot,plan,lotplan,area,feature_name,alias_name,parcel_type,cover_type
0,28663085,None,None,None,0.0,None,Durian Street Jonquil Court,Unlinked parcel or inter,Base
1,28663092,34,RP816756,34RP816756,781.0,None,None,Lot Type Parcel,Base
2,28663100,26,RP816756,26RP816756,715.0,None,None,Lot Type Parcel,Base
3,28663111,12,RP816755,12RP816755,763.0,None,None,Lot Type Parcel,Base
4,28663124,21,RP816754,21RP816754,753.0,None,None,Lot Type Parcel,Base


In [59]:
''' 
----------------------------------
Prepare the parcels to council land relational dataframe
----------------------------------
'''
# Import the Council Property to DCDB Base lots relational list
parcels_to_land = pd.DataFrame.spatial.from_table(os.path.join(input_gdb_path, 'INPUT_TB_DCDB_CouncilPropertyRel')).drop(['OBJECTID'], axis=1)

# Select only the required column
parcels_land = parcels_to_land.loc[:, ('segpar', 'relationships', 'landnos')].rename({'relationships' : 'rels'}, axis=1)

# Split the land numbers string into list
parcels_land.landnos = parcels_land.landnos.apply(lambda v: [int(x) for x in v.split(';')])

# Explode on the land numbers column (this will create duplicate parcel segpar records)
parcels_land = parcels_land.explode('landnos', ignore_index=True).rename({'landnos' : 'landno'}, axis=1)

# Inspect
parcels_land.head()

,segpar,rels,landno
0,28663092,1,146617
1,28460061,1,996564
2,32043083,1,145846
3,20484023,1,144907
4,20689095,1,112272


In [60]:
''' 
----------------------------------
Land Aggregates

Identify the land aggregates for each parcels
Land aggregates are where there are multiple council land parcels linked to a DCDB parcel
----------------------------------
'''

# Select only those parcels that have more than one relation
parcels_landagg = parcels_land.loc[parcels_land['rels'].astype(int) > 1]

# Group the land parcels by DCDB segpar
land_agg = parcels_landagg.loc[:, ('segpar', 'landno')].groupby(by='segpar').count().rename({'landno' : 'aggct'}, axis=1).reset_index()

# Create a unique aggregate id for the matched building unit plans
land_agg['aggid'] = land_agg.apply(lambda row: f'LND{row.name + 1:06}', axis=1)

# Joint the agg ids back onto the parcles
parcels_landagg = pd.merge(parcels_landagg, land_agg, on='segpar', how='left').drop(['rels'], axis=1)

# Inspect
parcels_landagg.head()

,segpar,landno,aggct,aggid
0,20354142,302263,6,LND000026
1,20354142,302271,6,LND000026
2,20354142,302272,6,LND000026
3,20354142,302262,6,LND000026
4,20354142,302264,6,LND000026


In [61]:
''' 
----------------------------------
BUP Aggregates

Identify the building unit plan aggregates for each parcel
----------------------------------
'''

# Import the building unit plan table
bup = pd.DataFrame.spatial.from_table(os.path.join(input_gdb_path, 'INPUT_TB_DCDB_BuildingUnitPlans')).drop(['OBJECTID'], axis=1)

# Join the BUP table to parcles and filter out non-joins
parcels_bup = pd.merge(parcels[['segpar', 'lotplan']], bup, on='lotplan', how='left').dropna()

# Create the building unit plan to parcels aggregate table
bup_agg = pd.merge(
    parcels_bup[['bup_plan', 'segpar']].groupby(by='bup_plan').count(),
    parcels_bup[['bup_plan', 'lot_area_am']].groupby(by='bup_plan').sum(),
    on='bup_plan',
    how='left'
).rename({'segpar' : 'aggct'}, axis=1).reset_index()

# Create a unique aggregate id for the matched building unit plans
bup_agg['aggid'] = bup_agg.apply(lambda row: f'BUP{row.name + 1:06}', axis=1)

# Join the building unit plan aggregates back onto the parcles layer
parcels_bup = pd.merge(parcels_bup[['segpar', 'bup_plan']].drop_duplicates(), bup_agg, on='bup_plan', how='left')

# Inspect
parcels_bup.head()

,segpar,bup_plan,aggct,lot_area_am,aggid
0,54498346,BUP102683,96,18036.0,BUP000004
1,54498399,SP106800,36,6508.0,BUP000027
2,30517321,SP171395,11,1287.0,BUP000045
3,30517345,SP206319,36,9652.0,BUP000052
4,30519141,SP277494,114,17874.0,BUP000072


In [62]:
''' 
----------------------------------
Named Aggregates

Identify the named aggregates for each parcel (based on the DCDB alias name)
----------------------------------
'''

# Get the named exclusions from the input vars table as list
alias_excl = pd.read_excel(vars_xlsx, 'DCDB_AliasExclude')['alias_name'].tolist()

# Extract a subset of the parcels table for only those with an alias name
parcels_named = parcels[['segpar', 'alias_name']].dropna()

# Drop those rows that have an excluded alias name
parcels_named = parcels_named.loc[~parcels_named['alias_name'].isin(alias_excl)]

# Group the parcels by name
named_agg = parcels_named.groupby(by='alias_name').count().rename({'segpar' : 'aggct'}, axis=1)

# Drop all the rows with a count less than 2
named_agg = named_agg.loc[named_agg['aggct'] > 1].reset_index()

# Create a unique aggregate id for the matched alias names
named_agg['aggid'] = named_agg.apply(lambda row: f'NME{row.name + 1:06}', axis=1)

# Join the named aggregates back onto the parcels (dropping non-joins)
parcels_named = pd.merge(parcels_named[['segpar', 'alias_name']].drop_duplicates(), named_agg, on='alias_name', how='left').dropna().reset_index(drop=True)

# Set the agg id and count columns to integer
parcels_named['aggct'] = parcels_named['aggct'].astype('Int64')

# Inspect
parcels_named.head()


,segpar,alias_name,aggct,aggid
0,55042099,Greenwich Court Sherwood Court,2,NME001071
1,28678072,West Mount Cotton Road,2,NME001168
2,28690056,Redland Bay Road,25,NME001124
3,54496155,CMS41085,4,NME000681
4,54498213,CMS23092,76,NME000300


In [63]:
''' 
----------------------------------
Holding Aggregates

Identify the holding aggregates for each parcel
----------------------------------
'''

# Property holdings
land_holdings = land[['landno', 'propno']].drop_duplicates()

# Group the parcels by holding
holdings_agg = land_holdings.groupby(by='propno').count().rename({'landno' : 'aggct'}, axis=1)

# Drop all the rows with a count less than 2
holdings_agg = holdings_agg.loc[holdings_agg['aggct'] > 1].reset_index()

# Create a unique aggregate id for the property holdings
holdings_agg['aggid'] = holdings_agg.apply(lambda row: f'HLD{row.name + 1:06}', axis=1)

# Join the holding aggregates back onto the land holdings (dropping non-joins)
holdings_agg = pd.merge(land_holdings, holdings_agg, on='propno', how='left').dropna().reset_index(drop=True)

# Set the agg count columns to integer
holdings_agg['aggct'] = holdings_agg['aggct'].astype('Int64')

# Get a copy of the parcels table
parcels_holding = parcels_land.loc[:, ('segpar', 'landno')]

# Join the holding aggregate id and count onto the parcel records
parcels_holding = pd.merge(parcels_holding, holdings_agg, on='landno', how='left').dropna().reset_index(drop=True)

# Set the joined columns to integer
parcels_holding['propno'] = parcels_holding['propno'].astype('Int64')
parcels_holding['aggct'] = parcels_holding['aggct'].astype('Int64')

# Inspect
parcels_holding.loc[parcels_holding['aggid'] == 'HLD000525']

,segpar,landno,propno,aggct,aggid
0,20616088,102308,8778,13,HLD000525
1,20616096,102286,8778,13,HLD000525
1626,20616095,102292,8778,13,HLD000525
2961,20616077,102324,8778,13,HLD000525
3066,20616092,102300,8778,13,HLD000525
3256,20616087,102307,8778,13,HLD000525
3722,20616085,102315,8778,13,HLD000525
3765,20616084,102319,8778,13,HLD000525
4202,20616094,102294,8778,13,HLD000525
4313,20616091,102305,8778,13,HLD000525


In [64]:
''' 
----------------------------------
Address

Identify the address aggregates for each parcel

NOTE: Address aggregates are now redundant and the outputs of this cell will not be used
----------------------------------
'''

# Addresses
land_address = land.loc[:, ('landno', 'unit_nos', 'house_nos', 'street', 'locality', 'postcode')].rename({
    'unit_nos' : 'unit', 
    'house_nos' : 'house',
}, axis=1)

# List of address columns
address_cols = ['house_from', 'house_to', 'street', 'locality', 'postcode']

# Function to get house from and to details and strip the non numeric characters
def houseFromTo(n, c='-'):
    if n:
        ft = n.split(c)
        f = int(re.sub(r'[^0-9.]', '', ft[0]))
        t = int(re.sub(r'[^0-9.]', '', ft[len(ft) - 1]))
        return[f, t]
    else:
        return [0, 0]

# Create a house from and to column using the house number and remove all non integer characters
land_address.loc[:,'house_from'] = land_address.loc[:,'house'].apply(lambda x: houseFromTo(x)[0])
land_address.loc[:,'house_to'] = land_address.loc[:,'house'].apply(lambda x: houseFromTo(x)[1])

# Create a new concatenated address column
land_address['address'] = \
    land_address['house_from'].astype(str) + '-' + \
    land_address['house_from'].astype(str) + ', ' + \
    land_address['street'].astype(str) + ', ' + \
    land_address['locality'].astype(str) + ' ' + \
    land_address['postcode'].astype(str)

# Group the land parcels by address
address_agg = land_address.loc[:, ('landno', 'address')].groupby(by='address').count().rename({'landno' : 'aggct'}, axis=1)

# Drop all the rows with a count less than 2
address_agg = address_agg.loc[address_agg['aggct'] > 1].reset_index()

# Create a unique aggregate id for the property address aggregates
address_agg['aggid'] = address_agg.apply(lambda row: f'ADD{row.name + 1:06}', axis=1)

# Join the address aggregates back onto the land address table (dropping non-joins)
address_agg = pd.merge(land_address, address_agg, on='address', how='left').dropna(subset=['aggct', 'aggid']).reset_index(drop=True)

# Set the agg id column to integer
address_agg['aggct'] = address_agg['aggct'].astype('Int64')

# Get a copy of the parcels table
parcels_address = parcels_land.loc[:, ('segpar', 'landno')]

# Join the address aggregate id and count onto the parcel records
parcels_address = pd.merge(parcels_address, address_agg, on='landno', how='left').dropna(subset=['aggct', 'aggid']).reset_index(drop=True)

# Inspect
parcels_address.loc[parcels_address['aggid'] == 'ADD000653'].sort_values(by='segpar')

,segpar,landno,unit,house,street,locality,postcode,house_from,house_to,address,aggct,aggid
10716,20335087,201386,None,122-124,Mooloomba Road,Point Lookout,4183,122.0,124.0,"122-122, Mooloomba Road, Point Lookout 4183",11,ADD000653
1771,20335088,201386,None,122-124,Mooloomba Road,Point Lookout,4183,122.0,124.0,"122-122, Mooloomba Road, Point Lookout 4183",11,ADD000653
14130,20335089,201376,1,122-124,Mooloomba Road,Point Lookout,4183,122.0,124.0,"122-122, Mooloomba Road, Point Lookout 4183",11,ADD000653
12076,20335090,201377,2,122-124,Mooloomba Road,Point Lookout,4183,122.0,124.0,"122-122, Mooloomba Road, Point Lookout 4183",11,ADD000653
8297,20335091,201378,3,122-124,Mooloomba Road,Point Lookout,4183,122.0,124.0,"122-122, Mooloomba Road, Point Lookout 4183",11,ADD000653
19143,20335092,201379,4,122-124,Mooloomba Road,Point Lookout,4183,122.0,124.0,"122-122, Mooloomba Road, Point Lookout 4183",11,ADD000653
14875,20335093,201380,5,122-124,Mooloomba Road,Point Lookout,4183,122.0,124.0,"122-122, Mooloomba Road, Point Lookout 4183",11,ADD000653
6541,20335094,201381,6,122-124,Mooloomba Road,Point Lookout,4183,122.0,124.0,"122-122, Mooloomba Road, Point Lookout 4183",11,ADD000653
11603,20335095,201382,7,122-124,Mooloomba Road,Point Lookout,4183,122.0,124.0,"122-122, Mooloomba Road, Point Lookout 4183",11,ADD000653
11964,20335096,201383,8,122-124,Mooloomba Road,Point Lookout,4183,122.0,124.0,"122-122, Mooloomba Road, Point Lookout 4183",11,ADD000653


In [65]:

''' 
----------------------------------
Import Shared Boundaries
----------------------------------
'''

# Import the DCDB Boundaries feature class as a dataframe
parcel_bounds = pd.DataFrame.spatial.from_featureclass(os.path.join(input_gdb_path, 'INPUT_FC_DCDB_SharedBoundaries')).drop(['OBJECTID', 'SHAPE'], axis=1)

# Insepct
parcel_bounds.head()

,segpar,shared_segpar,shared_length,shared_type,shared_name
0,28663092,28663086,18.25,Road,Jonquil Court
1,28663092,28663086,1.75,Road,Jonquil Court
2,28663092,28663093,38.84,Lot,None
3,28663092,28663255,9.44,Lot,None
4,28663092,28663254,10.56,Lot,None


In [66]:
''' 
----------------------------------
Common Property

Identify the common property aggregates for each parcel
----------------------------------
'''

# Get property use table
propuse = pd.read_excel(vars_xlsx, 'PropertyUse_to_DevGroup')

# Merge property use tags onto council land
land_propuse = pd.merge(land.loc[:, ('landno', 'propno', 'prop_use')], propuse.loc[:, ('prop_use', 'tag')], on='prop_use', how='left').dropna(subset=['tag'])

# Reset property number to be an integer
land_propuse['propno'] = land_propuse['propno'].astype('Int64')

# Link the property use to parcels via the parcels to council land mapping table
parcels_propuse = pd.merge(parcels_land, land_propuse, on='landno', how='left')

# Select all common properties from the data frame
common_props = parcels_propuse.loc[parcels_propuse['tag'] == 'Common'].copy()

# Get all parcels that share a boundary with the common properties
common_props['touching_parcels'] = common_props['segpar'].apply(lambda segpar: set(parcel_bounds.loc[parcel_bounds['shared_segpar'] == segpar]['segpar'].to_list()))

# --------------------------
# BUP

# Get the bup aggerate id linked to the common property
common_props['bup_aggid'] = common_props['segpar'].apply(lambda segpar: set(parcels_bup.loc[parcels_bup['segpar'] == segpar]['aggid'].to_list()))

# Get the linked parcels for the bup aggregates
common_props['bup_parcels'] = common_props['bup_aggid'].apply(lambda aggids : \
    set().union(*[set(parcels_bup.loc[parcels_bup['aggid'] == aggid]['segpar'].to_list()) for aggid in aggids if len(aggid) > 0]) \
)

# --------------------------
# Named

# Get the named aggerate id for the common property
common_props['named_aggid'] = common_props['segpar'].apply(lambda segpar: set(parcels_named.loc[parcels_named['segpar'] == segpar]['aggid'].to_list()))

# Get the linked parcels for the named aggregates
common_props['named_parcels'] = common_props['named_aggid'].apply(lambda aggids : \
    set().union(*[set(parcels_named.loc[parcels_named['aggid'] == aggid]['segpar'].to_list()) for aggid in aggids if len(aggid) > 0]) \
)

# --------------------------
# Holding

# Get the holding aggerate id linked to the common property
common_props['holding_aggid'] = common_props['segpar'].apply(lambda segpar: set(parcels_holding.loc[parcels_holding['segpar'] == segpar]['aggid'].to_list()))

# Get the linked parcels for the holding aggregates
common_props['holding_parcels'] = common_props['holding_aggid'].apply(lambda aggids : \
    set().union(*[set(parcels_holding.loc[parcels_holding['aggid'] == aggid]['segpar'].to_list()) for aggid in aggids if len(aggid) > 0]) \
)

# --------------------------
# Land

# Get the address aggerate id for the common property
common_props['land_aggid'] = common_props['segpar'].apply(lambda segpar: set(parcels_landagg.loc[parcels_landagg['segpar'] == segpar]['aggid'].to_list()))

# Get the linked parcels for the address aggregates
common_props['land_parcels'] = common_props['land_aggid'].apply(lambda aggids : \
    set().union(*[set(parcels_landagg.loc[parcels_landagg['aggid'] == aggid]['segpar'].to_list()) for aggid in aggids if len(aggid) > 0]) \
)

# --------------------------

# Combile all linked aggregate parcels into single field
common_props['agg_parcels'] = common_props.apply(lambda row: set().union(*[row.bup_parcels, row.named_parcels, row.holding_parcels, row.land_parcels]), axis=1)

# Drop redundant columns
common_props = common_props.drop(['bup_aggid', 'bup_parcels', 'named_aggid', 'named_parcels', 'holding_aggid', 'holding_parcels', 'land_aggid', 'land_parcels'], axis=1)

# Drop any of the touching parcels that do not share a common aggregate
common_props['linked_parcels'] = common_props.apply(lambda row: row.touching_parcels.intersection(row.agg_parcels), axis=1)

# Keep necessary columns
common_props = common_props.loc[:, ('segpar', 'linked_parcels')]

# --------------------------

# Inset the segpar into the linked parcels for completeness
common_props['linked_parcels'] = common_props.apply(lambda row: [*row.linked_parcels.union([row.segpar])], axis=1)

# Sort values
common_props = common_props.sort_values(by='segpar')

# Inspect
common_props.head(10)

,segpar,linked_parcels
61027,19789099,"[19789097, 19789098, 19789099]"
51434,19790170,"[19790225, 19790236, 19790237, 19790238, 19790..."
12906,19790195,"[19790208, 19790209, 19790210, 19790211, 19790..."
45981,19790196,"[19790212, 19790213, 19790214, 19790215, 19790..."
20309,19790197,"[19790208, 19790209, 19790210, 19790211, 19790..."
82291,19790214,"[19790213, 19790214, 19790215, 19790195, 19790..."
32551,19790225,"[19790225, 19790226, 19790227, 19790228, 19790..."
61390,19790255,"[19790094, 19790095, 19790096, 19790097, 19790..."
19384,19790256,"[19790094, 19790095, 19790096, 19790097, 19790..."
13224,19790257,"[19790150, 19790151, 19790152, 19790153, 19790..."


In [67]:
''' 
----------------------------------
Compile common property aggregates
----------------------------------
'''
# --------------------------------

'''
The following function moves through the grouped parcels and creates a list
of final aggregates that account for when a parcel falls within multiple groups.
In this case all of the groups and parcels that are related by one or many 
shared parcles are pulled together to form a final 'aggregate of aggregates' 
grouping. This process handles duplicate parcel 'segpar' identifiers by making 
use of python sets which cannot contain duplicate entries.
'''

def compileAggregates(series):

    # Define a container for the list of final aggregates
    final_aggregates = []

    # First pass run through of aggregate groups
    for id, group in series.items():
        
        g = set(group)

        for i, agg in enumerate(final_aggregates):
            if len(agg.intersection(g)) > 0:
                final_aggregates[i] = agg.union(g)
                break
        else:
            final_aggregates.append(g)

    # A procedure to contain temporary aggregate groupings
    temp_aggregates = []

    # Second pass run through of the aggregates
    while True:

        for agg in final_aggregates:
            for i, temp_agg in enumerate(temp_aggregates):
                if len(temp_agg.intersection(agg)) > 0:
                    temp_aggregates[i] = temp_agg.union(agg)
                    break
            else:
                temp_aggregates.append(agg)

        if len(temp_aggregates) == len(final_aggregates):
            break
        else:
            final_aggregates = temp_aggregates[:]
            temp_aggregates = []

    return final_aggregates

# Group parcles by aggregate id (creates series)
parcels_common_comp = compileAggregates(common_props['linked_parcels'])

# Create final dataframe from the processed sets
parcels_common = pd.DataFrame(list(zip(
        [f'CMN{i + 1:06}' for i, v in enumerate(parcels_common_comp)], # Create aggregate IDs
        [[*x] for x in parcels_common_comp] # Convert segpar ids list of sets to list of lists
    )), columns=['aggid', 'segpar'])

# Explode on segpar
parcels_common = parcels_common.explode('segpar').reset_index(drop=True)

# Inspect
parcels_common.head()

,aggid,segpar
0,CMN000001,19789097
1,CMN000001,19789098
2,CMN000001,19789099
3,CMN000002,19790094
4,CMN000002,19790095


In [68]:
''' 
----------------------------------
Create a mapping of all aggregates for use down stream
----------------------------------
'''

# Concatenate all aggregate dataframes
all_aggregates = pd.concat([
    parcels_bup.loc[:,('segpar','aggid')],
    parcels_named.loc[:,('segpar','aggid')],
    parcels_holding.loc[:,('segpar','aggid')],
    parcels_landagg.loc[:,('segpar','aggid')],
    parcels_common.loc[:, ('segpar', 'aggid')]
])

# Aggregate type helper function
def getAggType(idstr):

    if idstr[:3] == 'LND':
        return 'land'
    if idstr[:3] == 'BUP':
        return 'bup'
    if idstr[:3] == 'HLD':
        return 'holding'
    if idstr[:3] == 'NME':
        return 'named'
    if idstr[:3] == 'CMN':
        return 'common'
    else:
        return None

# Extract the aggregate type by id
all_aggregates['aggtype'] = all_aggregates.loc[:,('aggid')].apply(lambda x: getAggType(x))

# Inspect
all_aggregates.tail()

,segpar,aggid,aggtype
9704,63215611,CMN001038,common
9705,63215612,CMN001038,common
9706,63215613,CMN001038,common
9707,63215614,CMN001038,common
9708,63215615,CMN001038,common


In [69]:
''' 
----------------------------------
Process all aggregates and allocate a single aggregate id for each parcel
----------------------------------
'''

# Group parcles by aggregate id (creates series)
agg_groups = all_aggregates.groupby(by='aggid')['segpar'].apply(list)

# Define a container for the list of final aggregates
final_aggregates = compileAggregates(agg_groups)

# Create an aggregates dataframe from the processed sets
aggregates = pd.DataFrame(list(zip(
        [f'AGG{i + 1:06}' for i, v in enumerate(final_aggregates)], # Create aggregate IDs
        [[*x] for x in final_aggregates] # Convert segpar ids list of sets to list of lists
    )), columns=['aggid', 'segpar'])

# Inspect
aggregates.head()

,aggid,segpar
0,AGG000001,"[30525356, 30525357, 30525358, 30525359, 30525..."
1,AGG000002,"[30514282, 30514283, 30514284, 30514285, 30514..."
2,AGG000003,"[20327178, 20327179, 20327180, 20327181]"
3,AGG000004,"[54498400, 54498401, 54498403, 54498345, 54498..."
4,AGG000005,"[28401045, 28401046, 28401047, 28401048]"


In [70]:
''' 
----------------------------------
Filter aggregates by type and output final list
----------------------------------
'''

# Get parcel list (as set) for each aggregate type
lnd_ls = {*all_aggregates.loc[all_aggregates['aggtype'] == 'land'].loc[:,('segpar')].to_list()}
bup_ls = {*all_aggregates.loc[all_aggregates['aggtype'] == 'bup'].loc[:,('segpar')].to_list()}
hld_ls = {*all_aggregates.loc[all_aggregates['aggtype'] == 'holding'].loc[:,('segpar')].to_list()}
nme_ls = {*all_aggregates.loc[all_aggregates['aggtype'] == 'name'].loc[:,('segpar')].to_list()}
cmn_ls = {*all_aggregates.loc[all_aggregates['aggtype'] == 'common'].loc[:,('segpar')].to_list()}

# Get parcel list for pure named aggregates (name only - no other linked aggregates)
pure_nme_ls = nme_ls.difference(set().union(*[lnd_ls, bup_ls, hld_ls, cmn_ls]))

# Explode the aggregates by parcel
parcels_agg = aggregates.explode('segpar', ignore_index=True)

# Discard pure named aggregates
parcels_agg = parcels_agg[~parcels_agg['segpar'].isin(set().union(*[pure_nme_ls]))]

# Inspect
parcels_agg.head()

,aggid,segpar
0,AGG000001,30525356
1,AGG000001,30525357
2,AGG000001,30525358
3,AGG000001,30525359
4,AGG000001,30525360


In [71]:
''' 
----------------------------------
Prepare aggregate details table
----------------------------------
'''

# Join the all aggregates table to parcels land on segpar
aggregate_details = pd.merge(parcels_land, all_aggregates, on='segpar', how='left').dropna(subset=['aggid']).drop(['rels', 'aggid'], axis=1)

# Join in the parcels agg table to filter out redundant aggregates
aggregate_details = pd.merge(aggregate_details, parcels_agg, on='segpar', how='left').dropna(subset=['aggid'])

# Inspect
aggregate_details.head()

,segpar,landno,aggtype,aggid
0,28460061,996564,named,AGG000583
1,28460061,996564,common,AGG000583
2,20616088,102308,holding,AGG001546
3,20616096,102286,holding,AGG001546
4,32123008,138955,holding,AGG003043


In [72]:
''' 
----------------------------------
Export tables to geodatabase
----------------------------------
'''

# Write the aggregate details table back into the geodatabase
aggregate_details.spatial.to_table(os.path.join(input_gdb_path, 'GEN_TB_DCDB_AggregateDetails'), overwrite=True)

# Write the final aggregates back into the geodatabase
parcels_agg.spatial.to_table(os.path.join(input_gdb_path, 'GEN_TB_DCDB_Aggregates'), overwrite=True)

'G:\\Shared drives\\PIESolutions_03_Projects\\J000111 - Redlands planning assumption update\\06_Working Documents\\00_GIS Directory\\00_Data\\gam_model_run\\redland_gam_inputs.gdb\\GEN_TB_DCDB_Aggregates'